<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    CROSS BORDER FLOWS
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Raw Data Downloading Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to download the raw data for the Cross Border Flows Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError  # Import relevant exceptions
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

 <div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    2. ENTSO-E RESTful API.
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Connecting with the ENTSO-E API Tool.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- To donwload the neeeded data using the API tool, is mandatory to use a token autentication to connect and make the future request of data.
</div>

In [2]:
client = EntsoePandasClient(api_key='61e5bbbb-7e80-4540-a471-bd993873aa74')

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    3. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [3]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    4. Usefull Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- Indicate the year of all data is referring to in the variable data_year.
<br>
- The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>
- Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [4]:
# Year to which data refers to:
data_year = 2017
data_year_1 = data_year + 1

In [5]:
# Additional string to be appended
additional_path = "/RawData/CrossBorderFlows/"
additional_path_1 = "/RawData/CrossBorderFlows/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
cross_border_flows_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
cross_border_flows_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1

print("cross_border_flows_folder_path:", cross_border_flows_folder_path)
print("cross_border_flows_raw_data_folder_path:", cross_border_flows_raw_data_folder_path)

cross_border_flows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/
cross_border_flows_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/


In [6]:
# Define other parameters
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    5. Country List Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the list of countries according to the available data. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- Just those countries that interchange flows with other countries different of the ones modelled in Dispa-SET are defined in the list.
</div>

In [7]:
# Define a list of country codes
cross_border_flows_per_unit_country_list = ["AL", "BA", "BG", "BY", "EE", "GR", "FI", "HR", "HU", "IT", "LT", "LV",
                                            "MD", "ME", "MK", "MT", "PL", "RO", "RU", "RS", "SK", "TR", "UA"]

In [8]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(cross_border_flows_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(cross_border_flows_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(cross_border_flows_per_unit_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
cross_border_flows_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [9]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"cross_border_flows_folder_path:                            {cross_border_flows_folder_path}")   
print (f"cross_border_flows_raw_data_folder_path:                   {cross_border_flows_raw_data_folder_path}")
print (f"cross_border_flows_country_list_file:                      {cross_border_flows_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2017
cross_border_flows_folder_path:                            /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/
cross_border_flows_raw_data_folder_path:                   /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/
cross_border_flows_country_list_file:                      /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the sub-folders where all the cross border flows raw data is saved. 
</div>

In [10]:
# Convert data_year to string if it's not already
data_year = str(data_year)

# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the column 'Country_From' exists
if 'Country_From' not in df.columns:
    raise ValueError("Column 'Country_From' does not exist in the CSV file")

# Define the base directory where subfolders will be created
base_directory = os.path.join(cross_border_flows_raw_data_folder_path, data_year)

# Create a list to hold the paths of the created subfolders
country_folder_paths = []

# Create subfolders and save their paths
for country in df['Country_From']:
    # Create the subfolder path
    subfolder_path = os.path.join(base_directory, country)
    
    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Append the subfolder path to the list
    country_folder_paths.append(subfolder_path)

# Add the new column 'Country_Folder' to the DataFrame
df['Country_Folder'] = country_folder_paths

# Save the updated DataFrame back to the CSV file
df.to_csv(cross_border_flows_country_list_file, index=False)

print(f"Updated CSV file saved to '{cross_border_flows_country_list_file}' with new subfolders created.")

Updated CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv' with new subfolders created.


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the neighbor countries. 
</div>

In [11]:
# Raw data as a multiline string
data = """
GR
HR
MK, RS, TR
LT
RU
AL, MK, TR
RU
BA, RS
RS, UA
MT, ME
BY, RU
RU
RO
IT
BG, GR
IT
UA
MD, RS, UA
EE, FI, LV, LT
BG, HR, HU, RO
UA
BG, GR
HU, PL, RO, SK
"""

# Split the data into lines
lines = data.strip().split("\n")

# Initialize a list to hold the data
data_list = []

# Process each line
for line in lines:
    if line.strip() == "":
        # If the line is empty, add 11 empty strings
        data_list.append([""] * 4)
    else:
        neighbors = line.split(", ")
        data_list.append(neighbors)

# Create the DataFrame for new data
new_df = pd.DataFrame(data_list, columns=[f"Neighbor_{i}" for i in range(1, 5)])

# Path to the existing CSV file
existing_csv_file_path = cross_border_flows_country_list_file

# Read the existing CSV file into a DataFrame
existing_df = pd.read_csv(existing_csv_file_path)

# Concatenate the existing DataFrame with the new DataFrame horizontally
combined_df = pd.concat([existing_df, new_df], axis=1)

# Save the combined DataFrame back to the CSV file
combined_df.to_csv(existing_csv_file_path, index=False)
print(f"Data appended to '{existing_csv_file_path}' as new columns")

Data appended to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv' as new columns


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    6. Raw Data Download
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Donwloading the cross border flows raw data. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- The crows border flow data is downloaded in separate files for each country.
<br>
- Since the Acronym of Grece in the downloaded data is 'GR' and the Dispa-SET format for the country is 'EL'. All the needed changes in the used variables are done.
<br>
- The downloaded files will be joined into a single csv file under the name of the country which the flow comes from.
<br>
- The headers of these joined csv files are changed accordign the Dispa-SET cross border flow data format e.g. BE -> DE
</div>

In [12]:
# Define the start and end timestamps using the data_year variable
start = pd.Timestamp(f'{data_year}0101', tz='Europe/Brussels')
end = pd.Timestamp(f'{data_year_1}0101', tz='Europe/Brussels')

# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder'] + [f'Neighbor_{i}' for i in range(1, 5)]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_code_from = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Iterate through each neighbor column
    for neighbor_col in [f'Neighbor_{i}' for i in range(1, 5)]:
        country_code_to = row[neighbor_col]
        
        # Skip if the neighbor field is empty
        if pd.isna(country_code_to):
            continue
        
        try:
            # Query crossborder flows
            net_transfer = client.query_crossborder_flows(
                country_code_from, country_code_to, start=start, end=end
            )
            
            # Convert the index to a column
            net_transfer = net_transfer.reset_index()
            
            # Define the output file path
            output_file = os.path.join(country_folder, f'{country_code_to}.csv')
            
            # Save the DataFrame to a CSV file, including the index as a column
            net_transfer.to_csv(output_file, index=False)

            print(f"Data for {country_code_from} to {country_code_to} saved to '{output_file}'")

        except InvalidBusinessParameterError:
            print(f"Invalid business parameter for {country_code_from} to {country_code_to}. This pair may not be supported by the API.")
        
        except NoMatchingDataError:
            print(f"No matching data for {country_code_from} to {country_code_to} for the period {start} to {end}. Skipping.")
        
        except Exception as e:
            print(f"An error occurred while processing data for {country_code_from} to {country_code_to}: {e}")

print("All data has been processed and saved.")

Data for AL to GR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/GR.csv'
Data for BA to HR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BA/HR.csv'
Data for BG to MK saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/MK.csv'
Data for BG to RS saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/RS.csv'
Data for BG to TR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/TR.csv'
Data for BY to LT saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BY/LT.csv'
Data for EE to RU saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EE/RU.csv'
Data for GR to AL saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/GR/AL.csv'
Data for GR to MK saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Dat

In [13]:
# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Replace 'GR' with 'EL' in the entire DataFrame
df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)

# Save the updated DataFrame back to the CSV file
df.to_csv(cross_border_flows_country_list_file, index=False)

print(f"Replacements made and file saved: {cross_border_flows_country_list_file}")

Replacements made and file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv


/tmp/ipykernel_1100939/4086083268.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)


In [14]:
# Walk through the directory tree
for root, dirs, files in os.walk(cross_border_flows_raw_data_folder_path, topdown=False):
    # Rename files
    for name in files:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_file_path = os.path.join(root, name)
            new_file_path = os.path.join(root, new_name)
            os.rename(old_file_path, new_file_path)
            print(f"Renamed file: {old_file_path} to {new_file_path}")

    # Rename directories
    for name in dirs:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_dir_path = os.path.join(root, name)
            new_dir_path = os.path.join(root, new_name)
            os.rename(old_dir_path, new_dir_path)
            print(f"Renamed directory: {old_dir_path} to {new_dir_path}")

Renamed file: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/MK/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/MK/EL.csv
Renamed file: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/EL.csv
Renamed file: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/TR/GR.csv to /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/TR/EL.csv
Renamed directory: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/GR to /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EL


In [15]:
# Function to replace 'GR' with 'EL'
def replace_gr_with_el(lst):
    return ['EL' if x == 'GR' else x for x in lst]

# Applying the function to both lists
data = replace_gr_with_el(data)
cross_border_flows_per_unit_country_list = replace_gr_with_el(cross_border_flows_per_unit_country_list)

# Print the updated lists
print("Updated data list:", data)
print("Updated cross_border_flows_per_unit_country_list:", cross_border_flows_per_unit_country_list)

Updated data list: ['\n', 'G', 'R', '\n', 'H', 'R', '\n', 'M', 'K', ',', ' ', 'R', 'S', ',', ' ', 'T', 'R', '\n', 'L', 'T', '\n', 'R', 'U', '\n', 'A', 'L', ',', ' ', 'M', 'K', ',', ' ', 'T', 'R', '\n', 'R', 'U', '\n', 'B', 'A', ',', ' ', 'R', 'S', '\n', 'R', 'S', ',', ' ', 'U', 'A', '\n', 'M', 'T', ',', ' ', 'M', 'E', '\n', 'B', 'Y', ',', ' ', 'R', 'U', '\n', 'R', 'U', '\n', 'R', 'O', '\n', 'I', 'T', '\n', 'B', 'G', ',', ' ', 'G', 'R', '\n', 'I', 'T', '\n', 'U', 'A', '\n', 'M', 'D', ',', ' ', 'R', 'S', ',', ' ', 'U', 'A', '\n', 'E', 'E', ',', ' ', 'F', 'I', ',', ' ', 'L', 'V', ',', ' ', 'L', 'T', '\n', 'B', 'G', ',', ' ', 'H', 'R', ',', ' ', 'H', 'U', ',', ' ', 'R', 'O', '\n', 'U', 'A', '\n', 'B', 'G', ',', ' ', 'G', 'R', '\n', 'H', 'U', ',', ' ', 'P', 'L', ',', ' ', 'R', 'O', ',', ' ', 'S', 'K', '\n']
Updated cross_border_flows_per_unit_country_list: ['AL', 'BA', 'BG', 'BY', 'EE', 'EL', 'FI', 'HR', 'HU', 'IT', 'LT', 'LV', 'MD', 'ME', 'MK', 'MT', 'PL', 'RO', 'RU', 'RS', 'SK', 'TR', 'UA

In [16]:
# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the column 'Country_Folder' exists
if 'Country_Folder' not in df.columns:
    raise ValueError("Column 'Country_Folder' does not exist in the CSV file")

# Function to join CSV files in a directory
def join_csv_files_in_directory(directory_path):
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    if not csv_files:
        return None
    
    # Read all CSV files into DataFrames
    dataframes = {csv_file: pd.read_csv(os.path.join(directory_path, csv_file)) for csv_file in csv_files}
    
    # Find the CSV file with the largest number of rows
    largest_file = max(dataframes, key=lambda x: len(dataframes[x]))
    base_df = dataframes[largest_file].iloc[:, :2].copy()
    base_df.columns = [base_df.columns[0], largest_file.replace('.csv', '')]
    
    # Merge the other CSV files based on the first column
    for csv_file, df in dataframes.items():
        if csv_file == largest_file:
            continue
        temp_df = df.iloc[:, [0, 1]]
        temp_df.columns = [temp_df.columns[0], csv_file.replace('.csv', '')]
        base_df = pd.merge(base_df, temp_df, on=base_df.columns[0], how='left')
    
    return base_df

# Create a new column for the paths of the new CSV files
df['Country_File_Path'] = ''

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_folder = row['Country_Folder']
    
    # Join CSV files in the directory
    joined_df = join_csv_files_in_directory(country_folder)
    
    if joined_df is not None:
        # Define the output file path
        output_file = os.path.join(country_folder, f"{os.path.basename(country_folder)}.csv")
        
        # Save the joined DataFrame to a new CSV file
        joined_df.to_csv(output_file, index=False)
        
        # Update the DataFrame with the path of the new CSV file
        df.at[index, 'Country_File_Path'] = output_file

        print(f"Joined CSV file saved to '{output_file}'")

# Save the updated DataFrame back to the main CSV file
df.to_csv(cross_border_flows_country_list_file, index=False)

print("All data has been processed and saved.")

Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/AL.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BA/BA.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/BG.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BY/BY.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EE/EE.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EL/EL.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/FI/FI.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HR/HR.csv'
Joined CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HU/

In [17]:
# Read the main CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the required columns exist
if 'Country_From' not in df.columns or 'Country_File_Path' not in df.columns:
    raise ValueError("The CSV file must contain 'Country_From' and 'Country_File_Path' columns.")

# Function to update the headers of a CSV file
def update_csv_headers(file_path, new_header_prefix):
    # Read the CSV file into a DataFrame
    csv_df = pd.read_csv(file_path)
    
    # Get the current headers
    current_headers = csv_df.columns.tolist()
    
    # Create new headers for columns from the second column onward
    new_headers = [current_headers[0]] + [f"{new_header_prefix} -> {col}" for col in current_headers[1:]]
    
    # Update the DataFrame with the new headers
    csv_df.columns = new_headers
    
    # Save the updated DataFrame back to the CSV file
    csv_df.to_csv(file_path, index=False)
    print(f"Updated headers in '{file_path}'")

# Iterate through each row in the main DataFrame
for index, row in df.iterrows():
    country_from = row['Country_From']
    country_file_path = row['Country_File_Path']
    
    # Check if the file path is not empty and exists
    if pd.notna(country_file_path) and os.path.exists(country_file_path):
        update_csv_headers(country_file_path, country_from)
    else:
        print(f"File path '{country_file_path}' does not exist or is empty. Skipping...")

print("All CSV files have been processed.")

Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/AL.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BA/BA.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/BG.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BY/BY.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EE/EE.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EL/EL.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/FI/FI.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HR/HR.csv'
Updated headers in '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HU/HU.csv'
Updated headers in '/home/ray/Dispa-SET_Unleas

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    7. Raw Data Format
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Addapting the time step data to the UTC for all the countries.
</div>

In [18]:
# Read the country list CSV file
country_list_df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Define the function to convert time to UTC
def convert_to_utc(time_str):
    local_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S%z')
    utc_time = local_time.astimezone(pytz.utc)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S%z')

# Process each CSV file
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue

    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the 'index' column exists
    if 'index' not in df.columns:
        print(f"'index' column not found in file: {file_path}")
        continue

    # Convert the 'index' column to UTC
    df['index'] = df['index'].apply(convert_to_utc)
    
    # Save the updated CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")

Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/AL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BA/BA.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/BG.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BY/BY.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EE/EE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EL/EL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/FI/FI.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HR/HR.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HU/HU.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData

In [19]:
# Read the country list CSV file
country_list_df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Function to update the year in the 'index' column
def update_index_year(df, data_year):
    # Ensure the 'index' column exists
    if 'index' not in df.columns:
        raise ValueError("'index' column not found in DataFrame")
    
    # Update the year in the 'index' column
    df['index'] = df['index'].apply(lambda x: f"{data_year}{x[4:]}" if str(x)[:4] != str(data_year) else x)
    
    return df

# Process each CSV file specified in the 'Country_File_Path' column
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure there are enough rows to move the first four rows to the last
    if len(df) < 4:
        print(f"Not enough rows to process in file: {file_path}")
        continue
    
    # Extract the first four rows (excluding headers)
    first_four_rows = df.iloc[:4].copy()
    
    # Drop the first four rows from the DataFrame
    df = df.iloc[4:].reset_index(drop=True)
    
    # Append the first_four_rows to the end of the DataFrame
    df = pd.concat([df, first_four_rows]).reset_index(drop=True)
    
    # Update the 'index' column year
    df = update_index_year(df, data_year)
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")


Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/AL/AL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BA/BA.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BG/BG.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/BY/BY.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EE/EE.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/EL/EL.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/FI/FI.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HR/HR.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2017/HU/HU.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    7. Cross Border Flows Clean File
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Joining all the cros border flows data to a single csv file with named as the analized year.
</div>

In [20]:
# Read the country list CSV file
country_list_df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Process each CSV file specified in the 'Country_File_Path' column
file_paths = country_list_df['Country_File_Path'].dropna().tolist()

# Identify the CSV file with the largest number of rows
max_rows = 0
base_df = None
for file_path in file_paths:
    # Ensure the file exists
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if len(df) > max_rows:
            max_rows = len(df)
            base_df = df.copy()

# If no base_df was found, raise an error
if base_df is None:
    raise ValueError("No valid CSV files found.")

# Initialize the combined DataFrame with the first column from the base DataFrame
combined_df = pd.DataFrame(base_df.iloc[:, 0])
combined_df.columns = [base_df.columns[0]]  # Keep the original name of the first column

# Add data from each CSV file to the combined DataFrame
for file_path in file_paths:
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        # Merge the data based on the first column
        combined_df = pd.merge(combined_df, df, on=base_df.columns[0], how='left')

# Save the combined DataFrame to a new CSV file named after the data_year variable
output_file_path = os.path.join(cross_border_flows_folder_path, f"{data_year}.csv")
combined_df.to_csv(output_file_path, index=False)
print(f"Combined CSV file saved: {output_file_path}")

Combined CSV file saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/2017.csv


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Dividing the clean data in time stepts of 15 minutes, 30 minutes, and 1 hour.
</div>

In [21]:
csv_file_path = os.path.join(cross_border_flows_folder_path, f'{data_year}.csv')

# Create the new directories
intervals = ['1h', '30min', '15min']
for interval in intervals:
    os.makedirs(os.path.join(cross_border_flows_folder_path, interval), exist_ok=True)

# Read the original CSV file
df = pd.read_csv(csv_file_path)

# Convert the 'index' column to datetime
df['index'] = pd.to_datetime(df['index'], format='%Y-%m-%d %H:%M:%S%z')

# Function to extract rows at a specific time step and save to a new CSV file
def extract_and_save(df, interval, folder_name):
    # Resample the DataFrame
    resampled_df = df.set_index('index').resample(interval).first().reset_index()
    
    # Define the new file path
    new_file_path = os.path.join(cross_border_flows_folder_path, folder_name, f'{data_year}.csv')
    
    # Save the resampled DataFrame to the new CSV file
    resampled_df.to_csv(new_file_path, index=False)
    print(f"File saved: {new_file_path}")

# Extract and save rows at different time steps
extract_and_save(df, '1H', '1h')
extract_and_save(df, '30T', '30min')
extract_and_save(df, '15T', '15min')

File saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/1h/2017.csv


/tmp/ipykernel_1100939/2621242742.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()
/tmp/ipykernel_1100939/2621242742.py:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/30min/2017.csv


/tmp/ipykernel_1100939/2621242742.py:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.set_index('index').resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/15min/2017.csv


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Copying the time already formated Cross Border Flows data to the main Dispa-SET data base dirtectory
</div>

In [22]:
additional_path_2 = "/Database/CrossBorderFlows/"

# Construct the power_plants_raw_data_folder_path variable
cross_border_flows_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_2

In [23]:
# Define the subfolder names
subfolders = ['1h', '30min', '15min']

# Function to copy files
def copy_files(data_year, source_base_path, dest_base_path, subfolders):
    for subfolder in subfolders:
        source_path = os.path.join(source_base_path, subfolder, f"{data_year}.csv")
        dest_folder_path = os.path.join(dest_base_path, subfolder)

        # Create the destination subfolder if it does not exist
        os.makedirs(dest_folder_path, exist_ok=True)

        dest_path = os.path.join(dest_folder_path, f"{data_year}.csv")
        
        # Copy the file
        if os.path.isfile(source_path):
            shutil.copy2(source_path, dest_path)
            print(f"Copied {source_path} to {dest_path}")
        else:
            print(f"File {source_path} does not exist")

# Call the function
copy_files(data_year, cross_border_flows_folder_path, cross_border_flows_data_base_folder_path, subfolders)

Copied /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/1h/2017.csv to /home/ray/Dispa-SET_Unleash/Database/CrossBorderFlows/1h/2017.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/30min/2017.csv to /home/ray/Dispa-SET_Unleash/Database/CrossBorderFlows/30min/2017.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/15min/2017.csv to /home/ray/Dispa-SET_Unleash/Database/CrossBorderFlows/15min/2017.csv
